In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import matplotlib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import multiprocessing as mp
import backtesting as btng
from strategy import ARSIstrat
from typing import List, Union 
from tqdm import tqdm
from PERMUTATIONS import get_permutation

In [32]:
new_df = pd.read_csv('BATS_QQQ, 60_a45be.csv')
start = '2018-01-01'
end_excl = '2024-01-01'   # exclusive upper bound (covers all of 2018–2022)
new_df['time'] = pd.to_datetime(new_df['time'].astype(int), unit='s')
new_df.set_index('time', inplace=True)
new_df.sort_index(inplace=True)
new_df = new_df.loc[(new_df.index >= start) & (new_df.index < end_excl)].copy()
new_df = new_df.rename(columns={
    'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'
})

bt = Backtest(new_df, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True)    
stats = bt.run()
print(stats)
bt.plot() 

Backtest.run:   0%|          | 0/10342 [00:00<?, ?bar/s]

Start                     2018-01-02 14:30:00
End                       2023-12-29 20:30:00
Duration                   2187 days 06:00:00
Exposure Time [%]                    92.32666
Equity Final [$]                3648285.77783
Equity Peak [$]                 3835892.58312
Return [%]                          264.82858
Buy & Hold Return [%]               156.99849
Return (Ann.) [%]                    24.12742
Volatility (Ann.) [%]                24.66463
CAGR [%]                              16.0807
Sharpe Ratio                          0.97822
Sortino Ratio                         1.82102
Calmar Ratio                          0.86662
Alpha [%]                           207.92781
Beta                                  0.36243
Max. Drawdown [%]                   -27.84098
Avg. Drawdown [%]                    -1.43266
Max. Drawdown Duration      325 days 01:00:00
Avg. Drawdown Duration        7 days 19:00:00
# Trades                                 1398
Win Rate [%]                      

/Users/matteogiori/trycloneinsider/TradingBot/.venv313/lib/python3.13/site-packages/backtesting/_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '8h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p3955', ...)

In [33]:
stats, heatmap = bt.optimize(
        n_fast=range(5, 16),          # 5..15
        n_slow=range(10, 41, 2),      # 10..40 step 2
        n_vslow=range(40, 121, 5),    # 40..120 step 5 (the "very slow" you asked to include)
        maximize='Sharpe Ratio',
        return_heatmap=True,
        constraint=lambda p: p.n_fast < p.n_slow < p.n_vslow,
        #method='random', max_tries=1500, random_state=42,  # uncomment to speed up search
    )

Sharpe_Ratio, Return, Trades = stats[['Sharpe Ratio', 'Return [%]', '# Trades']].values
print(stats[['Sharpe Ratio', 'Return [%]', '# Trades']])


/Users/matteogiori/trycloneinsider/TradingBot/.venv313/lib/python3.13/site-packages/backtesting/backtesting.py:1624: UserWarning: Searching for best of 2777 configurations.
  output = _optimize_grid()
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable 

Backtest.optimize:   0%|          | 0/2777 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-

Backtest.run:   0%|          | 0/10342 [00:00<?, ?bar/s]

Sharpe Ratio      1.218982
Return [%]      429.018694
# Trades              1354
dtype: object


In [34]:
best = stats._strategy
BNF = best.n_fast
BNS = best.n_slow
BNVS = best.n_vslow
params = {
    'n_fast':  int(BNF),
    'n_slow':  int(BNS),
    'n_vslow': int(BNVS),
    'sig_len': int(getattr(best, 'sig_len', 14)),
    'sl_pct':  float(getattr(best, 'sl_pct', 0.01)),
}

print(params)

{'n_fast': 9, 'n_slow': 22, 'n_vslow': 55, 'sig_len': 14, 'sl_pct': 0.01}


In [36]:
Sharpe_Ratio_OPTI_real, Return_OPTI_real, Trades = stats[['Sharpe Ratio', 'Return [%]', '# Trades']].values
print("In-sample Sarpe Ratio", Sharpe_Ratio_OPTI_real, "Best n_fast", BNF, "Best n_slow", BNS, "Best n_vslow", BNVS)


n_permutations = 1000
perm_better_count = 1
permuted_pfs = []
print("In-Sample MCPT")

for perm_i in tqdm(range(1, n_permutations)):

    train_perm = get_permutation(new_df)

    train_bt = Backtest(train_perm, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True) 

    stat_perm, _ = train_bt.optimize(
        n_fast=range(5, 16),          # 5..15
        n_slow=range(10, 41, 2),      # 10..40 step 2
        n_vslow=range(40, 121, 5),    # 40..120 step 5 (the "very slow" you asked to include)
        maximize='Sharpe Ratio',
        return_heatmap=True,
        constraint=lambda p: p.n_fast < p.n_slow < p.n_vslow,
    )

    best_SR_perm, _, _ = stat_perm[['Sharpe Ratio', 'Return [%]', '# Trades']].values

    if best_SR_perm >= Sharpe_Ratio_OPTI_real:
        perm_better_count += 1

    permuted_pfs.append(best_SR_perm)

insample_mcpt_pval = perm_better_count / n_permutations
print(f"In-sample MCPT P-Value: {insample_mcpt_pval}")

plt.style.use('dark_background')
pd.Series(permuted_pfs).hist(color='blue', label='Permutations')
plt.axvline(Sharpe_Ratio_OPTI_real, color='red', label='Real')
plt.xlabel("Profit Factor")
plt.title(f"In-sample MCPT. P-Value: {insample_mcpt_pval}")
plt.grid(False)
plt.legend()
plt.show()

In-sample Sarpe Ratio 1.2189822993651378 Best n_fast 9 Best n_slow 22 Best n_vslow 55
In-Sample MCPT


  0%|          | 0/999 [00:00<?, ?it/s]/Users/matteogiori/trycloneinsider/TradingBot/.venv313/lib/python3.13/site-packages/backtesting/backtesting.py:1624: UserWarning: Searching for best of 2777 configurations.
  output = _optimize_grid()
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_mo

Backtest.optimize:   0%|          | 0/2777 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-

...

Backtest.optimize:   0%|          | 0/2777 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                       Open    High      Low    Close  Volume
time                                                         
2018-01-

Process SpawnPoolWorker-98:                             
Process SpawnPoolWorker-105:
Process SpawnPoolWorker-102:
Process SpawnPoolWorker-100:
Process SpawnPoolWorker-103:
Process SpawnPoolWorker-107:
Process SpawnPoolWorker-99:                             
Process SpawnPoolWorker-104:
Process SpawnPoolWorker-101:
Process SpawnPoolWorker-97:
Process SpawnPoolWorker-108:
Process SpawnPoolWorker-106:                            
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib

KeyboardInterrupt: 